In [14]:
library(rgdal)
library(raster)
library(dplyr)
library(RColorBrewer)

Loading required package: sp
rgdal: version: 1.2-18, (SVN revision 718)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.1.3, released 2017/20/01
 Path to GDAL shared files: /Library/Frameworks/R.framework/Versions/3.4/Resources/library/rgdal/gdal
 GDAL binary built with GEOS: FALSE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: /Library/Frameworks/R.framework/Versions/3.4/Resources/library/rgdal/proj
 Linking to sp version: 1.2-7 

Attaching package: ‘dplyr’

The following objects are masked from ‘package:raster’:

    intersect, select, union

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [52]:
# suppression 2D plots
path <- '/Users/dongmeichen/Documents/beetle/data'
df.fire <- read.csv(sprintf('%s/plot_2D_firedata.csv', path))
df.forest <- read.csv(sprintf('%s/plot_2D_forestdata.csv', path))
df <- cbind(df.fire, df.forest)
# head(df)

In [9]:
out <- '/Users/dongmeichen/Documents/beetle/images/DEA/'

In [54]:
vars <- c('SprsFires', 'PctSprs', 'SprsAcre', 'mStdAge', 'density', 'vcc', 'GAP1', 'GAP2', 'GAP3')
mains <- c('No. fires suppressed', 'Ratio of suppressed fires', 'Fire size of suppressed fires',
           'Stand age', 'Tree density', 'Vegetation condition class', 'Disturbance events allowed',
           'Disturbance events suppressed', 'Subject to human activities')
png(paste0(out,"suppression_2D_plots.png"), width=9, height=9, units="in", res=300)
par(mfrow=c(3,3),mar=c(3.5,3.5,3,1))
for(v in vars){
    if(v == 'GAP2'){
        plot(df[,paste0(v, '.y')] ~ df[, v], 
             type='l',
             lty=2,
             lwd=2,
             #cex.lab=0.8,
             cex.axis=1.5,
             main=mains[which(vars==v)],
             xlab='', 
             ylab='')                 
    }else{
        plot(df[,paste0(v, '.y')] ~ df[, v], 
             type='l',
             lty=1,
             lwd=2,
             #cex.lab=0.8,
             cex.axis=1.5,
             main=mains[which(vars==v)],
             xlab='', 
             ylab='')  
    }
    
}
dev.off()

pdf 
  2

In [35]:
# vegetation condition class barplots
df <- read.csv(paste0(path, "/mpb10km_with_beetle_data.csv"))

In [44]:
df <- df[complete.cases(df), ]

In [36]:
df$severity <- ifelse(df$prs >= 17 & df$mfri >= 16, 'replacement', ifelse(df$pls >= 17 & df$mfri <= 4, 'low', 'mixed'))
df$severity.no <- ifelse(df$severity == 'replacement', 3, ifelse(df$severity == 'low', 1, 2))

In [45]:
df.cpa <- df[df$SprsCPA != Inf, ]

In [37]:
get.df <- function(df, var1, var2, fun){
    if(var2 == 'mfri'){
        cond <- paste0(var2, ' %in% c(1:22)')
    }else if(var2 == 'vcc'){
        cond <- paste0(var2, ' %in% c(1:6)')
    }else if(var2 %in% c('pls', 'pms', 'prs')){
        cond <- paste0(var2, ' %in% c(1:20)')
    }else if(var2 == 'severity.no'){
        cond <- paste0(var2, ' %in% c(1:3)')
    }
    strings <- paste0('df %>% subset(', cond, ') %>% dplyr::select(') # forest==1 & 
    strings <- paste0(strings, var2, ',', var1, ') %>% group_by(', var2)
    strings <- paste0(strings,  ') %>% summarise(', fun, '=', fun, '(', var1)
    strings <- paste0(strings, ', na.rm=TRUE), grids = sum(!is.na(', var1)
    if(fun=='sum' & var1 %in% sprs.vars){
        strings <- paste0(strings,')), average=sum(', var1, ', na.rm=TRUE)/sum(!is.na(', var1)
        strings <- paste0(strings,')), percent=sum(!is.na(', var1, '))/length(',var1,'))')
    }else if(fun=='sum'){
        strings <- paste0(strings,')), average=sum(', var1, ', na.rm=TRUE)/sum(!is.na(', var1, ')))')
    }else{
        strings <- paste0(strings,')), percent=sum(!is.na(', var1, '))/length(',var1,'))')
    }
    sdf <- eval(parse(text=strings))
    return(sdf)
}

In [38]:
sprs.vars <- c('SprsCosts', 'SprsAcres', 'SprsCPA', 'SprsFires', 'PctSprs', 
               'SprsAcre', 'SprsDays', 'OutDays')
plt.vars <- c('beetleAcres', 'mStdAge', 'density', 'PctOld', sprs.vars)
sprs.titles <- c('Suppression costs', 'Suppression acres', 'Unit suppression costs',
                 'No. suppressed fires', 'Ratio of suppressed fires', 'Fire size suppressed',
                 'Containment duration', 'Fire out duration')
plt.titles <- c('MPB affected acres', 'Stand age', 'Tree density', 'Ratio of old trees', sprs.titles)

In [39]:
agr.vars <- rep('average', 12)
funs <- rep('sum', 12)

In [40]:
get.table <- function(df, var, agr.var, fun){
    df.low <- df[df$severity == 'low', ]
    df.mixed <- df[df$severity == 'mixed', ]
    df.replacement <- df[df$severity == 'replacement', ]
    sdf <- as.data.frame(get.df(df, var, 'vcc', fun=fun))
    sdf.low <- as.data.frame(get.df(df.low, var, 'vcc', fun=fun))
    sdf.mixed <- as.data.frame(get.df(df.mixed, var, 'vcc', fun=fun))
    sdf.replacement <- as.data.frame(get.df(df.replacement, var, 'vcc', fun=fun))
    sdf.group <- rbind(sdf.low, sdf.mixed, sdf.replacement)
    sdf.group$severity <- c(rep('low', dim(sdf.low)[1]), rep('mixed', dim(sdf.mixed)[1]), 
                            rep('replacement', dim(sdf.replacement)[1]))
    mat <- matrix(0, ncol=length(unique(sdf.group$vcc)), nrow=length(unique(sdf.group$severity)))
    rownames(mat) <- sort(unique(sdf.group$severity))
    colnames(mat) <- sort(unique(sdf.group$vcc))
    for (sev in sort(unique(sdf.group$severity))) {
        for (v in sort(unique(sdf.group$vcc))) {
            sev.sum <- sum(sdf.group[sdf.group$vcc == v, 'grids'])
            if(length(sdf.group[sdf.group$severity == sev & sdf.group$vcc == v, 'grids']) != 0){
                mat[sev, v] <- sdf.group[sdf.group$severity == sev & sdf.group$vcc == v, 'grids'] / sev.sum
            }  
        }
    }
    data <- mat * rbind(sdf[,agr.var], sdf[,agr.var],sdf[,agr.var])
    return(data)
}

In [41]:
get.plot.col <- function(df, title){
    barplot(df, col=c('#636363', '#bdbdbd', '#f0f0f0'), main = title, cex.names = 1.2, cex.lab=1.2, cex.axis=1.2, cex.main=1.5)
}

In [50]:
plot.vcc.severity <- function(){
    png(paste0(out, "vcc_severity_plots.png"), width=12, height=9, units="in", res=300)
    par(mfrow=c(3,4),xpd=FALSE,mar=c(3,3,3,0))
    for(var in plt.vars){
        if(var == 'SprsCPA'){
            dt <- get.table(df.cpa, var, agr.var=agr.vars[plt.vars==var], fun=funs[plt.vars==var])
        }else{
            dt <- get.table(df, var, agr.var=agr.vars[plt.vars==var], fun=funs[plt.vars==var])
        }
        get.plot.col(dt, plt.titles[plt.vars==var])
        #print(var)
    }
    dev.off()
}

In [51]:
plot.vcc.severity()

[1] "beetleAcres"
[1] "mStdAge"
[1] "density"
[1] "PctOld"
[1] "SprsCosts"
[1] "SprsAcres"
[1] "SprsCPA"
[1] "SprsFires"
[1] "PctSprs"
[1] "SprsAcre"
[1] "SprsDays"
[1] "OutDays"


pdf 
  2

In [76]:
v1 <- c('logSprsCPA.p1', 'SprsFires', 'PctSprs', 'SprsAcre', 'SprsDays', 'OutDays')
v2 <- c('PctLarge', 'PctOld')
v3 <- c('mStdAge', 'density', 'vcc', 'mfri', 'prs', 'pms', 'pls')
vs <- c(v1, v2, v3)

In [144]:
# point plots of two variables
df1 <- read.csv(sprintf('%s/mpb10km_data_clean.csv', path))
df2 <- read.csv(sprintf('%s/mpb10km_data_wo_costs.csv', path))
df3 <- read.csv(sprintf('%s/mpb10km_data_wo_firedata.csv', path))
df4 <- read.csv(sprintf('%s/mpb10km_data_wo_FIA.csv', path))

In [145]:
mains <- c('Unit suppression costs (log)', 'No. suppressed fires (log)', 'Ratio of suppressed fires', 
           'Fire size suppressed (log)', 'Containment duration (log)', 'Fire out duration (log)',
           'Ratio of large trees', 'Ratio of old trees', 'Stand age (log)', 'Tree density (log)',
           'Vegetation condition class', 'Mean fire return interval', 'Percent of replacement-severity fire',
           'Percent of mixed-severity fire', 'Percent of low-severity fire')

In [149]:
png(paste0(out, "simple_linear_models.png"), width=15, height=9, units="in", res=300)
par(mfrow=c(3,5),mar=c(3.5,3.5,3,1))
for(field in vs){
    if(field=='logSprsCPA.p1'){
        df <- df1[,c('beetleAcres', field)]
        #df[,field] <- exp(df[,field]) - 1
    }else if(field %in% v1[-1]){
        df <- df2[,c('beetleAcres', field)]
    }else if(field %in% v2){
        df <- df3[,c('beetleAcres', field)]
    }else if(field %in% v3){
        df <- df4[,c('beetleAcres', field)]
    }
    if(field %in% c('SprsFires', 'SprsAcre', 'SprsDays', 'OutDays', 'mStdAge', 'density')){
        df[, field] <- log(df[, field]+1)
    }
    if(field %in% c('vcc', 'mfri', 'prs', 'pms', 'pls')){
        plot(df[,'beetleAcres'] ~ df[, field],
         type='h',
         lwd=3,
         pch=16, 
         cex=0.8,
         col=rgb(0, 0, 0, 0.2),
         main=mains[which(vs==field)],
         xlab='',
         ylab='')
        #lines(lowess(df[,'beetleAcres'] ~ df[, field]), col=2, lwd=2, lty=2)   
    }
    if(!(field %in% c('vcc', 'mfri', 'prs', 'pms', 'pls'))){
        df$beetleAcres <- df$beetleAcres^0.05
        colnames(df)[2] <- 'field'
        g <- lm(beetleAcres~field, data=df)
        colnames(df)[2] <- field
        modsum <- summary(g)
        r2 = modsum$adj.r.squared
        my.p = modsum$coefficients[2,4]
        rp = vector('expression',2)
        rp[1] = substitute(expression(italic(R)^2 == MYVALUE), 
            list(MYVALUE = format(r2,dig=3)))[2]
        rp[2] = substitute(expression(italic(p) == MYOTHERVALUE), 
            list(MYOTHERVALUE = format(my.p, digits = 2)))[2]
        plot(df[,'beetleAcres'] ~ df[, field], 
             pch=16, 
             cex=0.8,
             col=rgb(0, 0, 0, 0.2),
             main=mains[which(vs==field)],
             xlab='',
             ylab='')
        lines(lowess(df[,'beetleAcres'] ~ df[, field]), col=2, lwd=2, lty=2)
        abline(g, lwd=2)
        if(field %in% c('mStdAge', 'density')){
            legend('bottomleft', legend = rp, bty = 'n')  
        }else{
            legend('bottomright', legend = rp, bty = 'n') 
        }  
    }     
}
dev.off()

pdf 
  2